In [1]:
%matplotlib inline

In [2]:
import glob
import gzip
import json

import numpy as np
import pandas as pd

In [3]:
run_title = "fasttext"

In [4]:
category_df = pd.read_csv("trec2019b_test_results_run_%s.csv" % run_title, dtype={
    "tweet_id": np.int64,
})
priority_df = pd.read_csv("trec2019b_test_results_priority_run_%s.csv" % run_title, dtype={
    "tweet_id": np.int64,
})

FileNotFoundError: [Errno 2] File trec2019b_test_results_run_fasttext.csv does not exist: 'trec2019b_test_results_run_fasttext.csv'

In [5]:
header_map = {
    "Donations": "CallToAction-Donations",
    "MovePeople": "CallToAction-MovePeople",
    "Volunteer": "CallToAction-Volunteer",
    "Advice": "Other-Advice",
    "ContextualInformation": "Other-ContextualInformation",
    "Discussion": "Other-Discussion",
    "Irrelevant": "Other-Irrelevant",
    "Sentiment": "Other-Sentiment",
    "CleanUp": "Report-CleanUp",
    "EmergingThreats": "Report-EmergingThreats",
    "Factoid": "Report-Factoid",
    "FirstPartyObservation": "Report-FirstPartyObservation",
    "Hashtags": "Report-Hashtags",
    "Location": "Report-Location",
    "MultimediaShare": "Report-MultimediaShare",
    "News": "Report-News",
    "NewSubEvent": "Report-NewSubEvent",
    "Official": "Report-Official",
    "OriginalEvent": "Report-OriginalEvent",
    "ServiceAvailable": "Report-ServiceAvailable",
    "ThirdPartyObservation": "Report-ThirdPartyObservation",
    "Weather": "Report-Weather",
    "GoodsServices": "Request-GoodsServices",
    "InformationWanted": "Request-InformationWanted",
    "SearchAndRescue": "Request-SearchAndRescue",
}

In [6]:
topic_num_to_id_map = {}

with open("../data/TRECIS-2019-B-Test.topics", "r") as in_file:
    topic_num = ""
    topic_id = ""
    
    for line in in_file:
        
        if line.strip() == "</top>":
            topic_num_to_id_map[topic_id] = topic_num
        
        if line.startswith("<num>"):
            topic_num = line.partition(">")[-1].partition("<")[0]
              
        if line.startswith("<dataset>"):
            topic_id = line.partition(">")[-1].partition("<")[0]

In [7]:
topic_num_to_id_map

{'albertaWildfires2019': 'TRECIS-CTIT-H-Test-029',
 'cycloneKenneth2019': 'TRECIS-CTIT-H-Test-030',
 'philippinesEarthquake2019': 'TRECIS-CTIT-H-Test-031',
 'coloradoStemShooting2019': 'TRECIS-CTIT-H-Test-032',
 'southAfricaFloods2019': 'TRECIS-CTIT-H-Test-033',
 'sandiegoSynagogueShooting2019': 'TRECIS-CTIT-H-Test-034'}

In [8]:
tweet_id_to_topic_map = {}
tweet_id_to_count_map = {}

for file_path in glob.iglob("../data/2019/*2019*B-test*.json.gz"):
    print(file_path)
    counter = 1
    with gzip.open(file_path, "rb") as in_file:
        for line_ in in_file:
            line = line_.decode("utf8")
            tweet_entry = json.loads(line)
            
            tweet_id = np.int64(tweet_entry["allProperties"]["id"])
            tweet_topic = tweet_entry["topic"]
            
            if ( '1100409564803956744' == tweet_entry["allProperties"]["id"] ):
                print("Found")
                print(tweet_entry["allProperties"]["id"], tweet_id, tweet_entry["allProperties"]["id"] == tweet_id)
            
            tweet_id_to_topic_map[tweet_id] = topic_num_to_id_map[tweet_topic]
            tweet_id_to_count_map[tweet_id] = counter
            
            counter += 1

../data/2019/trecis2019-B-test.coloradoStemShooting2019.json.gz
../data/2019/trecis2019-B-test.southAfricaFloods2019.json.gz
../data/2019/trecis2019-B-test.cycloneKenneth2019.json.gz
../data/2019/trecis2019-B-test.philippinesEarthquake2019.json.gz
../data/2019/trecis2019-B-test.sandiegoSynagogueShooting2019.json.gz
../data/2019/trecis2019-B-test.albertaWildfires2019.json.gz


In [9]:
len(tweet_id_to_count_map), priority_df.shape[0]

(13916, 15000)

In [10]:
category_map = {row["tweet_id"]: header_map[row["label"]] for idx, row in category_df.iterrows()}
priority_map = {row["tweet_id"]: row["priority"] for idx, row in priority_df.iterrows()}

In [11]:
category_df["inc_id"] = category_df["tweet_id"].apply(tweet_id_to_topic_map.get)
category_df["count"] = category_df["tweet_id"].apply(tweet_id_to_count_map.get)
category_df["priority"] = category_df["tweet_id"].apply(priority_map.get)

In [12]:
priority_scorer = {
    1 : 0.75,
    0 : 0.25
}

with open("nyu-smapp_run_%s.csv" % run_title, "w") as out_file:
    for row in category_df.drop_duplicates(subset="tweet_id").itertuples():
        content = [
            row.inc_id, 
            "Q0", 
            row.tweet_id,
            row.count,
            priority_scorer[row.priority],
            str([header_map[row.label]]).replace("'", '"'),
            "nyu-smapp_%s" % run_title
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")